In [ ]:
!pip install dataprep by

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
from datetime import datetime
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
import missingno as msno
import statsmodels.stats.api as sms
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import levene
from scipy.stats import shapiro
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import scale
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.manifold import Isomap,TSNE
from sklearn.feature_selection import mutual_info_classif
from tqdm.notebook import tqdm
from scipy.stats import ttest_ind
import plotly.offline as pyo
import scipy.stats as stats
import pymc3 as pm
from dataprep.eda import *
from dataprep.eda import plot
from dataprep.eda import plot_diff
from dataprep.eda import plot_correlation
from dataprep.eda import plot_missing
import plotly.figure_factory as ff
from collections import Counter
import pandas_profiling as pp
from mpl_toolkits.basemap import Basemap
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import folium
from folium import plugins

### MAIN

In [ ]:
READING_MAIN = pd.read_csv("../input/connection-between-frp-and-plants/PROJECT_FRP_TYPE_OF_PLANTS.csv")

In [ ]:
READING_MAIN

In [ ]:
READING_MAIN.rename(columns={"AD":"plants"},inplace=True)

In [ ]:
READING_MAIN

In [ ]:
print(READING_MAIN["acq_date"].value_counts())

In [ ]:
print(READING_MAIN["plants"].value_counts())

### DATA TYPES

#### MAX FRP

In [ ]:
MAX_FPR_DATA = READING_MAIN[READING_MAIN["frp"] == READING_MAIN["frp"].max()]

In [ ]:
MAX_FPR_DATA

In [ ]:
MAP_FUNCTION = folium.Map(location=[38.9637,35.2433],
                  tiles='openstreetmap',
                  zoom_start=6.4)

folium.CircleMarker(
    location=[37.0248,27.7112],
    radius=20,
    popup=f"<b>MAXIMUM FRP -- <b>{MAX_FPR_DATA.plants.values}",
    tooltip="Click me",
    color='#FF0000',
    fill=True,
    fill_color='#FF0000',
).add_to(MAP_FUNCTION)

MAP_FUNCTION

#### MIN FRP DATA

In [ ]:
MIN_FPR_DATA = READING_MAIN[READING_MAIN["frp"] == READING_MAIN["frp"].min()]

In [ ]:
MIN_FPR_DATA

#### 29 JULY

In [ ]:
JULY_29_DATA = READING_MAIN[READING_MAIN["acq_date"] == "7/29/2021"]

In [ ]:
JULY_29_DATA.reset_index(drop=True,inplace=True)

In [ ]:
JULY_29_DATA

In [ ]:
print(JULY_29_DATA["plants"].value_counts())

In [ ]:
JULY_29_CONIFEROUS = JULY_29_DATA[JULY_29_DATA["plants"] == "İğne Yapraklı Ormanlar"]

JULY_29_CONIFEROUS.reset_index(drop=True,inplace=True)

In [ ]:
JULY_29_CONIFEROUS

In [ ]:
CONIFEROUS_LAT_LONG = JULY_29_CONIFEROUS[["latitude","longitude"]]

CONIFEROUS_LAT_LONG.latitude = CONIFEROUS_LAT_LONG.latitude.astype(float)
CONIFEROUS_LAT_LONG.longitude = CONIFEROUS_LAT_LONG.longitude.astype(float)

HEAT_CONIFEROUS = list(zip(CONIFEROUS_LAT_LONG.latitude,CONIFEROUS_LAT_LONG.longitude))

In [ ]:
MAP_FUNCTION = folium.Map(location=[38.9637,30.2433],
                  tiles='Stamen Terrain',
                  zoom_start=6.5)

plugins.MarkerCluster(CONIFEROUS_LAT_LONG).add_to(MAP_FUNCTION)

MAP_FUNCTION

#### JULY 29 BASED ON TIME

In [ ]:
JULY_29_SORTED_TIME = JULY_29_DATA.copy()

JULY_29_SORTED_TIME.sort_values(by=["acq_time"],ascending=True,inplace=True)

JULY_29_SORTED_TIME

In [ ]:
print(JULY_29_SORTED_TIME[JULY_29_SORTED_TIME["frp"] == JULY_29_SORTED_TIME["frp"].max()])

In [ ]:
figure = plt.figure(figsize=(25,14))
sns.lineplot(x=JULY_29_SORTED_TIME["acq_time"],y=JULY_29_SORTED_TIME["frp"])
plt.title("TIME-FRP / JULY 29")
plt.tight_layout()
plt.show()

#### MAIN DATA MAP

In [ ]:
MAIN_LAT_LONG = READING_MAIN[["latitude","longitude"]]

MAIN_LAT_LONG.latitude = MAIN_LAT_LONG.latitude.astype(float)
MAIN_LAT_LONG.longitude = MAIN_LAT_LONG.longitude.astype(float)

HEAT_MAIN_LAT_LONG = list(zip(MAIN_LAT_LONG.latitude,MAIN_LAT_LONG.longitude))

In [ ]:
MAP_FUNCTION = folium.Map(location=[38.9637,30.2433],
                  tiles='Stamen Terrain',
                  zoom_start=6.5)

plugins.MarkerCluster(MAIN_LAT_LONG).add_to(MAP_FUNCTION)

MAP_FUNCTION

In [ ]:
MAIN_CONIFEROUS = READING_MAIN[READING_MAIN["plants"] == "İğne Yapraklı Ormanlar"]

MAIN_CONIFEROUS.reset_index(drop=True,inplace=True)

MAIN_CONIFEROUS

In [ ]:
MAIN_CONIFEROUS_LAT_LONG = MAIN_CONIFEROUS[["latitude","longitude"]]

MAIN_CONIFEROUS_LAT_LONG.latitude = MAIN_CONIFEROUS_LAT_LONG.latitude.astype(float)
MAIN_CONIFEROUS_LAT_LONG.longitude = MAIN_CONIFEROUS_LAT_LONG.longitude.astype(float)

HEAT_MAIN_CONIFEROUS = list(zip(MAIN_CONIFEROUS_LAT_LONG.latitude,MAIN_CONIFEROUS_LAT_LONG.longitude))

In [ ]:
MAP_FUNCTION = folium.Map(location=[38.9637,30.2433],
                  tiles='Stamen Terrain',
                  zoom_start=6.5)

plugins.MarkerCluster(MAIN_CONIFEROUS_LAT_LONG).add_to(MAP_FUNCTION)

MAP_FUNCTION

In [ ]:
MAP_FUNCTION = folium.Map(location=[38.9637,30.2433],
                  tiles='Stamen Terrain',
                  zoom_start=6.5)

plugins.MarkerCluster(MAIN_LAT_LONG).add_to(MAP_FUNCTION)

plugins.AntPath(MAIN_CONIFEROUS_LAT_LONG).add_to(MAP_FUNCTION)

MAP_FUNCTION

In [ ]:
MAP_FUNCTION = folium.Map(location=[38.9637,30.2433],
                  tiles='Stamen Terrain',
                  zoom_start=6.5)

plugins.MarkerCluster(MAIN_LAT_LONG).add_to(MAP_FUNCTION)

plugins.HeatMap(MAIN_CONIFEROUS_LAT_LONG).add_to(MAP_FUNCTION)

MAP_FUNCTION

In [ ]:
MAP_FUNCTION = folium.Map(location=[38.9637,30.2433],
                  tiles='Stamen Terrain',
                  zoom_start=6.5)

plugins.MarkerCluster(MAIN_LAT_LONG).add_to(MAP_FUNCTION)

plugins.HeatMap(HEAT_CONIFEROUS).add_to(MAP_FUNCTION)

MAP_FUNCTION

### PLOT

In [ ]:
plt.style.use("dark_background")

In [ ]:
figure = plt.figure(figsize=(25,14))
sns.lineplot(x=JULY_29_DATA["plants"],y=JULY_29_DATA["frp"])
plt.title("PLANTS-FRP / JULY 29")
plt.tight_layout()
plt.show()

In [ ]:
figure = plt.figure(figsize=(25,14))
sns.lineplot(x=JULY_29_DATA["plants"],y=JULY_29_DATA["brightness"])
plt.title("PLANTS-BRIGHTNESS / JULY 29")
plt.tight_layout()
plt.show()